In [2]:
import pandas as pd
df = pd.read_csv('miami-housing.csv')
df.head()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


use the library panda to read the csv file

In [3]:
#remove lat, long, parcelno and month sold.
del df["LATITUDE"]
del df["LONGITUDE"]
del df["PARCELNO"]
del df["month_sold"]

PARCELNO is self-explanatory, it is just a random id number used to identified each houses and have no effects on the price.
The position of the house is express clearly with much more useful parameters like RAIL_DIST, OCEAN_DIST, CNTR_DIST,... In comparison, LONGITUDE and LATITUDE is not very useful in providing information on the position of the house.
month_sold is not used because we cannot possibly know when we sold the house before we sold it and therefore not very useful in prediction.

In [4]:
params = df[["LND_SQFOOT", "TOT_LVG_AREA", "SPEC_FEAT_VAL",  "RAIL_DIST",  "OCEAN_DIST", "WATER_DIST",  "CNTR_DIST",  "SUBCNTR_DI",  "HWY_DIST",  "age",  "avno60plus"]]
prices = df['SALE_PRC']

struct_quality = pd.get_dummies(df.structure_quality)
struct_quality.rename(columns = {1:'structure_quality_1', 2:'structure_quality_2',3:'structure_quality_3',4:'structure_quality_4',5:'structure_quality_5'}, inplace=True)
params = params.join(struct_quality.astype(int))
params

,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,structure_quality_1,structure_quality_2,structure_quality_3,structure_quality_4,structure_quality_5
0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,0,0,0,1,0
1,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,0,0,0,1,0
2,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,0,0,0,1,0
3,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,0,0,0,1,0
4,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13927,6780,967,6580,3844.5,20568.0,3252.4,22175.9,12150.1,917.4,16,0,0,0,0,1,0
13928,7500,1854,2544,3593.6,20791.9,3077.7,22375.1,12316.8,738.2,26,0,0,0,0,1,0
13929,8460,1271,2064,4143.2,20307.9,3588.4,20966.9,12433.0,743.7,16,0,0,0,0,1,0
13930,7500,1613,3136,3986.9,20542.6,3589.1,21475.6,12458.0,626.1,16,0,0,0,0,1,0


Only structure_quality is suitable for one-hot encoding since it have 5 value that have meaning not linearly related to each other. In other words, it is just a catagory field.

In [5]:
import statsmodels.api as sm

params = sm.add_constant(params)
model = sm.OLS(prices, params).fit()

model.params

const                  383070.463939
LND_SQFOOT                  3.815532
TOT_LVG_AREA              179.041568
SPEC_FEAT_VAL               2.764561
RAIL_DIST                   4.665842
OCEAN_DIST                 -4.210879
WATER_DIST                  1.139366
CNTR_DIST                  -2.780598
SUBCNTR_DI                 -0.624793
HWY_DIST                    4.816770
age                     -1608.201474
avno60plus             -85585.314247
structure_quality_1   -242366.412223
structure_quality_2   -171480.235354
structure_quality_3    799434.935569
structure_quality_4   -117423.869904
structure_quality_5    114906.045852
dtype: float64

In [6]:
model.pvalues

const                   0.000000e+00
LND_SQFOOT              5.123621e-40
TOT_LVG_AREA            0.000000e+00
SPEC_FEAT_VAL          4.329787e-111
RAIL_DIST               1.516348e-66
OCEAN_DIST             4.422861e-216
WATER_DIST              5.416568e-07
CNTR_DIST              2.065588e-120
SUBCNTR_DI              3.324151e-05
HWY_DIST                3.765916e-69
age                     3.545544e-71
avno60plus              5.933055e-13
structure_quality_1     1.230882e-78
structure_quality_2    7.673695e-100
structure_quality_3    4.397242e-114
structure_quality_4     3.445918e-52
structure_quality_5     3.815964e-45
dtype: float64

377698.0738840018

In [26]:
list_of_params = []
for x in params.columns:
    list_of_params.append((model.params[x], x))

list_of_params.sort()

forward_step_params = params[['const']]

for i in range(len(params.columns)):
    if (list_of_params[i][1] == 'const'):
        continue

    X = sm.add_constant(forward_step_params)
    model_of_forward_step = sm.OLS(prices, X).fit()

    if list_of_params[i][0] > model_of_forward_step.aic:
        break
    
    new_param = params[list_of_params[i][1]]
    forward_step_params.join(new_param)

forward_step_params

,const
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
13927,1.0
13928,1.0
13929,1.0
13930,1.0
